Adapted from `rlcard/examples/leduc_holdem_dqn_pytorch.py`

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('./rlcard'))

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
import torch
import rlcard
from DQNAgent import DQNAgent
from rlcard.agents import RandomAgent, CFRAgent
from rlcard.utils import set_global_seed, tournament
from rlcard.utils import Logger

In [4]:
set_global_seed(0)

In [5]:
eval_env = rlcard.make('leduc-holdem', config={'seed': 0})

In [6]:
dqn_agent = DQNAgent(
    eval_env.action_num,
    eval_env.state_shape[0],
    memory=2_000_000,
    lr=0.1,
    batch_size=128,
    update_every=64,
    eps_start=0.12,
    target_update=300,
    hidden_neurons=[64]
)

dqn_agent.load('models/leduc_holdem_dqn/step-1921384.pt')

In [7]:
random_agent = RandomAgent(action_num=eval_env.action_num)

In [8]:
cfr_agent = CFRAgent(eval_env, model_path='./models/leduc_holdem_cfr')
cfr_agent.load()

In [9]:
def eval(agent1, agent2):
    eval_env.set_agents([agent1, agent2])
    return tournament(eval_env, 100000)

In [10]:
# dqn vs dqn
eval(dqn_agent, dqn_agent)

[0.000705, -0.000705]

In [11]:
# dqn vs random
eval(dqn_agent, random_agent)

[0.54875, -0.54875]

In [12]:
# random vs dqn
eval(random_agent, dqn_agent)

[-0.54497, 0.54497]

In [13]:
# random vs random
eval(random_agent, random_agent)

[0.00508, -0.00508]

In [14]:
# dqn vs cfr
eval(dqn_agent, cfr_agent)

[0.0092, -0.0092]

In [15]:
# cfr vs dqn
eval(cfr_agent, dqn_agent)

[-0.003285, 0.003285]

In [16]:
# cfr vs cfr
eval(cfr_agent, cfr_agent)

[-0.006355, 0.006355]

In [17]:
# cfr vs random
eval(cfr_agent, random_agent)

[0.713935, -0.713935]

In [18]:
# random vs cfr
eval(random_agent, cfr_agent)

[-0.71044, 0.71044]